In [54]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.display.max_columns = None

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marketing-strategy-personalised-offer/sample.csv
/kaggle/input/marketing-strategy-personalised-offer/train_data.csv
/kaggle/input/marketing-strategy-personalised-offer/test_data.csv


In [55]:
train = pd.read_csv('/kaggle/input/marketing-strategy-personalised-offer/train_data.csv')
test = pd.read_csv('/kaggle/input/marketing-strategy-personalised-offer/test_data.csv')
sample_submission = pd.read_csv('/kaggle/input/marketing-strategy-personalised-offer/sample.csv')
sample_submission.head(), train.shape, test.shape

(   id Offer Accepted
 0   0             No
 1   1             No
 2   2             No
 3   3             No
 4   4             No,
 (12379, 31),
 (5305, 30))

In [56]:
train.columns

Index(['offer expiration', 'income_range', 'no_visited_Cold drinks',
       'travelled_more_than_15mins_for_offer', 'Restaur_spend_less_than20',
       'Marital Status', 'restaurant type', 'age',
       'Prefer western over chinese', 'travelled_more_than_25mins_for_offer',
       'travelled_more_than_5mins_for_offer', 'no_visited_bars', 'gender',
       'car', 'restuarant_same_direction_house', 'Cooks regularly',
       'Customer type', 'Qualification', 'is foodie', 'no_Take-aways',
       'Job/Job Industry', 'restuarant_opposite_direction_house',
       'has Children', 'visit restaurant with rating (avg)', 'temperature',
       'Restaur_spend_greater_than20', 'Travel Time', 'Climate',
       'drop location', 'Prefer home food', 'Offer Accepted'],
      dtype='object')

In [57]:
train.head()

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,travelled_more_than_5mins_for_offer,no_visited_bars,gender,car,restuarant_same_direction_house,Cooks regularly,Customer type,Qualification,is foodie,no_Take-aways,Job/Job Industry,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food,Offer Accepted
0,2days,₹100000 or More,4~8,1,less1,Married partner,4 star restaurant,36,0,0,1,less1,Female,NaN,0,1,Individual,Bachelors degree,0,1~3,Unemployed,0,0,4,67,less1,22,Spring,Location B,0,No
1,2days,₹87500 - ₹99999,4~8,0,4~8,Married partner,Take-away restaurant,50plus,0,0,1,never,Female,NaN,1,1,Individual,Bachelors degree,0,1~3,Unemployed,0,1,3,89,1~3,18,Summer,Location B,0,Yes
2,2days,₹87500 - ₹99999,less1,1,1~3,Single,Cold drinks,26,1,0,1,never,Female,Mazda5,1,0,Individual,Bachelors degree,1,1~3,Arts Design Entertainment Sports & Media,1,1,4,67,less1,7,Winter,Location A,1,Yes
3,10hours,₹37500 - ₹49999,less1,0,1~3,Single,Take-away restaurant,46,1,0,1,never,Male,NaN,1,0,Individual,Some college - no degree,1,1~3,Sales & Related,0,1,3,89,1~3,7,Summer,Location C,0,No
4,2days,₹100000 or More,never,1,1~3,Single,4 star restaurant,21,0,1,1,less1,Female,NaN,0,0,Individual,Some college - no degree,1,gt8,Sales & Related,1,0,3,40,less1,7,Summer,Location C,0,No


In [58]:
test.head()

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,travelled_more_than_5mins_for_offer,no_visited_bars,gender,car,restuarant_same_direction_house,Cooks regularly,Customer type,Qualification,is foodie,no_Take-aways,Job/Job Industry,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food
0,2days,₹37500 - ₹49999,1~3,1,1~3,Divorced,Take-away restaurant,50plus,0,1,1,less1,Male,NaN,0,0,Individual,Graduate degree (Masters or Doctorate),0,4~8,Sales & Related,1,1,4,89,less1,7,Summer,Location C,0
1,10hours,₹25000 - ₹37499,4~8,0,1~3,Unmarried partner,Cold drinks,21,1,0,1,gt8,Male,NaN,0,0,Individual,Some college - no degree,1,gt8,Computer & Mathematical,1,1,0,67,less1,18,Summer,Location A,1
2,2days,₹100000 or More,never,1,never,Single,Cold drinks,21,1,0,1,never,Female,NaN,0,1,Individual,High School Graduate,0,less1,Unemployed,1,0,4,67,never,7,Summer,Location C,0
3,2days,₹25000 - ₹37499,never,1,1~3,Married partner,Restaurant with pub,41,0,0,1,never,Female,NaN,0,0,Individual,Some college - no degree,1,4~8,Unemployed,1,1,0,40,less1,18,Spring,Location C,1
4,10hours,₹12500 - ₹24999,never,0,1~3,Single,2 star restaurant,31,1,0,1,never,Male,NaN,1,0,With Family,High School Graduate,1,1~3,Unemployed,0,0,0,89,1~3,18,Summer,Location B,1


In [59]:
# features = train.drop(['Offer Accepted'], axis=1)
# for i in features.columns:
#     print(i)
#     print(features[i].value_counts())
#     print('///')
#     print(test[i].value_counts())
#     print('##################################################')

In [60]:
# train.isna().sum()

In [61]:
# test.isna().sum()

### Feature Engineering (?)

In [62]:
for i in [ 'travelled_more_than_15mins_for_offer','travelled_more_than_25mins_for_offer', 'travelled_more_than_5mins_for_offer']:
    print(train[i].value_counts())

1    6999
0    5380
Name: travelled_more_than_15mins_for_offer, dtype: int64
0    10901
1     1478
Name: travelled_more_than_25mins_for_offer, dtype: int64
1    12379
Name: travelled_more_than_5mins_for_offer, dtype: int64


In [63]:
conditions = [
    (train['travelled_more_than_5mins_for_offer'] == 1)& (train['travelled_more_than_15mins_for_offer'] == 0),
    (train['travelled_more_than_15mins_for_offer'] == 1)& (train['travelled_more_than_25mins_for_offer'] == 0),
    (train['travelled_more_than_25mins_for_offer'] == 1)
]

values = [0, 1, 2]

train['travel_for_offer'] = np.select(conditions, values)
train['travel_for_offer'].value_counts()
#why 1353 instead of 1478???????
conditions = [
    (test['travelled_more_than_5mins_for_offer'] == 1)& (test['travelled_more_than_15mins_for_offer'] == 0),
    (test['travelled_more_than_15mins_for_offer'] == 1)& (test['travelled_more_than_25mins_for_offer'] == 0),
    (test['travelled_more_than_25mins_for_offer'] == 1)
]
test['travel_for_offer'] = np.select(conditions, values)
test['travel_for_offer'].value_counts()


1    2396
0    2334
2     575
Name: travel_for_offer, dtype: int64

In [64]:
train = train.drop([ 'travelled_more_than_15mins_for_offer','travelled_more_than_25mins_for_offer', 'travelled_more_than_5mins_for_offer'], axis=1)
test = test.drop([ 'travelled_more_than_15mins_for_offer','travelled_more_than_25mins_for_offer', 'travelled_more_than_5mins_for_offer'], axis = 1)

In [65]:
train = train.drop([ 'restuarant_opposite_direction_house'], axis=1)
test = test.drop([ 'restuarant_opposite_direction_house'], axis = 1)

### Dealing with Missing Values

In [66]:
train = train.drop(['car'], axis=1)
test = test.drop(['car'], axis=1)
features = train.drop(['Offer Accepted'], axis=1)
label = train['Offer Accepted']

from sklearn.impute import SimpleImputer
ssi = SimpleImputer(strategy = 'most_frequent')
features = ssi.fit_transform(features)
ntest = ssi.transform(test)

features = pd.DataFrame(features, columns = train.drop(['Offer Accepted'], axis=1).columns)
test = pd.DataFrame(test, columns = test.columns)
features.shape, test.shape

((12379, 26), (5305, 26))

In [67]:
features.columns, test.columns

(Index(['offer expiration', 'income_range', 'no_visited_Cold drinks',
        'Restaur_spend_less_than20', 'Marital Status', 'restaurant type', 'age',
        'Prefer western over chinese', 'no_visited_bars', 'gender',
        'restuarant_same_direction_house', 'Cooks regularly', 'Customer type',
        'Qualification', 'is foodie', 'no_Take-aways', 'Job/Job Industry',
        'has Children', 'visit restaurant with rating (avg)', 'temperature',
        'Restaur_spend_greater_than20', 'Travel Time', 'Climate',
        'drop location', 'Prefer home food', 'travel_for_offer'],
       dtype='object'),
 Index(['offer expiration', 'income_range', 'no_visited_Cold drinks',
        'Restaur_spend_less_than20', 'Marital Status', 'restaurant type', 'age',
        'Prefer western over chinese', 'no_visited_bars', 'gender',
        'restuarant_same_direction_house', 'Cooks regularly', 'Customer type',
        'Qualification', 'is foodie', 'no_Take-aways', 'Job/Job Industry',
        'has Children

## Preprocessing

In [68]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

income_range_cols = ['Less than ₹12500', '₹12500 - ₹24999', '₹25000 - ₹37499','₹37500 - ₹49999', '₹50000 - ₹62499', '₹62500 - ₹74999', '₹75000 - ₹87499', '₹87500 - ₹99999', '₹100000 or More']
no_cols=['never','less1','1~3', '4~8','gt8']
no_col_names = ['no_visited_Cold drinks', 'Restaur_spend_less_than20', 'no_Take-aways','Restaur_spend_greater_than20','no_visited_bars']
#
age_cols = ['below21','21', '26','31','36','41','46','50plus']
#
Qualification_cols =['Some High School','High School Graduate','Associates degree','Some college - no degree', 'Bachelors degree','Graduate degree (Masters or Doctorate)']
temperature_cols =['40','67','89']
tt_cols = ['7','10','14','18','22'] #'Travel Time'=
no_col_num=['0','1','2','3','4'] #visit restaurant with rating (avg)

oe = OrdinalEncoder(categories=[income_range_cols,no_cols,no_cols,age_cols,no_cols,Qualification_cols,no_cols,temperature_cols,no_cols,tt_cols,no_col_num], handle_unknown='use_encoded_value', unknown_value=-1)
oe1 = OrdinalEncoder(categories=[income_range_cols], handle_unknown='use_encoded_value', unknown_value=-1)
oe2 = OrdinalEncoder(categories=[no_cols,no_cols,no_cols,no_cols,no_cols], handle_unknown='use_encoded_value', unknown_value=-1)
oe3 = OrdinalEncoder(categories=[age_cols], handle_unknown='use_encoded_value', unknown_value=-1)
oe4 = OrdinalEncoder(categories=[Qualification_cols], handle_unknown='use_encoded_value', unknown_value=-1)
oe5 = OrdinalEncoder(categories=[temperature_cols], handle_unknown='use_encoded_value', unknown_value=-1)
oe6 = OrdinalEncoder(categories=[tt_cols], handle_unknown='use_encoded_value', unknown_value=-1)
oe7 = OrdinalEncoder(categories=[no_col_num], handle_unknown='use_encoded_value', unknown_value=-1)

ohe = OneHotEncoder(sparse=False,handle_unknown="ignore")

nominal = ['offer expiration', 'Marital Status', 'restaurant type', 'Prefer western over chinese',  'gender', 'restuarant_same_direction_house', 'Cooks regularly', 'Customer type', 'is foodie', 'Job/Job Industry', 'has Children', 'Climate', 'drop location', 'Prefer home food']
ordinal = ['income_range', 'no_visited_Cold drinks', 'Restaur_spend_less_than20', 'age', 'no_visited_bars', 'Qualification', 'no_Take-aways', 'temperature', 'Restaur_spend_greater_than20', 'Travel Time', 'visit restaurant with rating (avg)']


from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[('ohe', ohe, nominal),
                                     #('OE',oe, ordinal)
                        ('oe1', oe1, ['income_range']),
                        ('oe2', oe2, no_col_names),
                        ('oe3', oe3, ['age']),
                        ('oe4', oe4, ['Qualification']),
                        ('oe5', oe5, ['temperature']), 
                        ('oe6', oe6, ['Travel Time']),
                        ('oe7', oe7, ['visit restaurant with rating (avg)']), 
                                    ],remainder='passthrough')

In [69]:

ft = ct.fit_transform(features.astype('U'))
feature_trans = pd.DataFrame(ft)

tt = ct.transform(test.astype('U'))
test_trans = pd.DataFrame(tt)

feature_trans.shape, test_trans.shape

((12379, 73), (5305, 73))

In [70]:
feature_trans.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
count,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379.0,12379
unique,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,9.0,5.0,5.0,5.0,5.0,5.0,8.0,6.0,3.0,5.0,5.0,3
top,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,2.0,2.0,1.0,0.0,1.0,3.0,2.0,3.0,1.0,1
freq,6791.0,6791.0,11867.0,7392.0,7714.0,10275.0,12268.0,9737.0,10180.0,8908.0,10653.0,10038.0,6272.0,6272.0,6335.0,6335.0,9737.0,9737.0,6195.0,6195.0,6448.0,9197.0,10732.0,10760.0,6223.0,6223.0,12226.0,11733.0,12338.0,11893.0,12149.0,11018.0,12229.0,11435.0,12347.0,12098.0,12133.0,12156.0,12239.0,12163.0,12204.0,11537.0,11748.0,12214.0,12275.0,12196.0,11917.0,11304.0,10859.0,12144.0,10541.0,7234.0,7234.0,10213.0,8065.0,10231.0,6789.0,8955.0,9014.0,6311.0,6311.0,2010.0,3502.0,5369.0,4794.0,6152.0,5157.0,2602.0,4264.0,5720.0,2931.0,2567.0,5646


In [71]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
label = lb.fit_transform(label)
label = pd.DataFrame(label)
label.head()

,0
0,0
1,1
2,1
3,0
4,0


In [72]:
lb.classes_

array(['No', 'Yes'], dtype='<U3')

In [73]:
feature_trans.dtypes

0     object
1     object
2     object
3     object
4     object
       ...  
68    object
69    object
70    object
71    object
72    object
Length: 73, dtype: object

#### Balancing Data

In [74]:
train['Offer Accepted'].value_counts(), features.shape


(Yes    6994
 No     5385
 Name: Offer Accepted, dtype: int64,
 (12379, 26))

In [75]:
from imblearn.over_sampling import SMOTE
ros = SMOTE()
feature_trans, label = ros.fit_resample(feature_trans, label)

In [76]:
feature_trans.shape, label.value_counts()

((13988, 73),
 0    6994
 1    6994
 dtype: int64)

## Model Selection

In [77]:
from sklearn.model_selection import GridSearchCV

#### Logistic Regression

In [78]:
# from sklearn.linear_model import LogisticRegression
# grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
# log = LogisticRegression(random_state=0)
# lg = GridSearchCV(log,grid,cv=3).fit(feature_trans.astype('int64'), label)
# lg.best_params_

#### SVC

In [79]:
# from sklearn.svm import SVC
# grid = {'C': [0.1, 1, 10, 100, 1000],
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
#               'kernel': ['rbf']}
# sv = SVC(random_state=0)
# svc = GridSearchCV(sv, grid).fit(feature_trans.astype('int64'), label)

# svc.best_params_

#### KNN

In [80]:
# from sklearn.neighbors import KNeighborsClassifier
# grid = {"n_neighbors":range(1,50)}
# neigh = KNeighborsClassifier()
# nn = GridSearchCV(neigh,grid,cv=10).fit(feature_trans.astype('int64'), label)
# nn.best_params_

#### Decision Tree

In [81]:
# from sklearn.tree import DecisionTreeClassifier
# cart = DecisionTreeClassifier(criterion='gini')
# grid = {'max_features': ['auto', 'sqrt', 'log2'],
#               'max_depth' : [10,100,1000],
#         'min_samples_split' : [1,10, 100],
        
#              }
# cartsearch = GridSearchCV(cart, grid, cv=5).fit(feature_trans.astype('int64'), label)
# cartsearch.best_params_

#### RF

In [82]:
# from sklearn.ensemble import RandomForestClassifier
# freg = RandomForestClassifier(random_state=0)
# param_grid = [{'n_estimators':[10,100], 'max_features':[10,50,100]}]
# from sklearn.model_selection import GridSearchCV
# grid_search = GridSearchCV(freg, param_grid, cv=5, scoring='f1_micro', return_train_score=True)
# grid_search.fit(feature_trans.astype('int64'), label)
# grid_search.best_params_, grid_search.best_estimator_

In [83]:
# params = {#'n_estimator':5,
#         #'learning_rate': 0.05,
#         'max_depth': range (2, 50, 2)
#         }
# from xgboost import XGBClassifier
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# xgbc = XGBClassifier()
# xgbcgs = GridSearchCV(xgbc, params, cv=3, scoring='f1_micro', return_train_score=True)
# # xgbc.fit(xtrain, ytrain)
# # yhatxb=xgbc.predict(xtest)
# # f1_score(ytest, yhatxb, average='micro')
# xgbcgs.fit(feature_trans.astype('int64'), label)

In [84]:
# xgbcgs.best_params_

In [85]:
# test_models=[]
# from sklearn.linear_model import LogisticRegression
# test_models.append(LogisticRegression(C= 0.1, penalty= 'l2',random_state=0))
# from sklearn.svm import SVC
# test_models.append(SVC(C= 100, gamma= 0.001, kernel= 'rbf'))
# from sklearn.naive_bayes import GaussianNB
# test_models.append(GaussianNB())
# from sklearn.neighbors import KNeighborsClassifier
# test_models.append(KNeighborsClassifier(n_neighbors=45))
# from sklearn.tree import DecisionTreeClassifier
# test_models.append(DecisionTreeClassifier(max_depth= 100, max_features= 'sqrt', min_samples_split= 100))
# cart = DecisionTreeClassifier(random_state=0)
# from sklearn.ensemble import BaggingClassifier
# test_models.append(BaggingClassifier(base_estimator=cart,n_estimators=10, random_state=0))
# from sklearn.ensemble import RandomForestClassifier
# test_models.append(RandomForestClassifier(max_features= 50, n_estimators= 100,random_state=0))
# from xgboost import XGBClassifier
# test_models.append(XGBClassifier(learning_rate= 0.05, max_depth= 3, n_estimator= 5))

# from sklearn.model_selection import cross_val_score

# f1scores = [['model','maxf1', 'meanf1']]
# from sklearn.metrics import f1_score
# from sklearn.model_selection import train_test_split
# for j in test_models:
#     f1scores.append([j,0,0])
#     cvs = cross_val_score(j, feature_trans.astype('int64'), label, scoring="f1_micro", cv=3)
# #         test_model = j.fit(xtrain, ytrain)
# #         yhattm=test_model.predict(xtest)
#     f1scores.append([j, max(cvs),np.mean(cvs)])

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

[04:41:41] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[04:41:42] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[04:41:44] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
 

In [86]:
# mcaret = pd.DataFrame(f1scores[1:], columns=f1scores[0])
# mcaret

,model,maxf1,meanf1
0,"LogisticRegression(C=0.1, random_state=0)",0.000000,0.000000
1,"LogisticRegression(C=0.1, random_state=0)",0.595024,0.581213
2,"SVC(C=100, gamma=0.001)",0.000000,0.000000
3,"SVC(C=100, gamma=0.001)",0.604247,0.575424
4,GaussianNB(),0.000000,0.000000
5,GaussianNB(),0.584513,0.555621
6,KNeighborsClassifier(n_neighbors=45),0.000000,0.000000
7,KNeighborsClassifier(n_neighbors=45),0.601459,0.583215
8,"DecisionTreeClassifier(max_depth=100, max_feat...",0.000000,0.000000
9,"DecisionTreeClassifier(max_depth=100, max_feat...",0.569284,0.561768


In [87]:
from sklearn.ensemble import RandomForestClassifier
freg = RandomForestClassifier(max_features= 50, n_estimators= 100, random_state=0)
from xgboost import XGBClassifier
# freg = XGBClassifier(learning_rate= 0.05, max_depth= 4, n_estimator= 5)
freg.fit(feature_trans.astype('int64'), label)
sub = freg.predict(test_trans.astype('int64'))
subd = pd.DataFrame( sub, columns=['Offer Accepted'])
subd['Offer Accepted'] = subd['Offer Accepted'].apply(lambda x: 'Yes' if x==1 else 'No')
subd['id']=subd.index
subd

[04:41:45] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




,Offer Accepted,id
0,Yes,0
1,Yes,1
2,No,2
3,No,3
4,Yes,4
...,...,...
5300,Yes,5300
5301,Yes,5301
5302,No,5302
5303,Yes,5303


In [88]:
submis= pd.DataFrame( subd, columns=['id', 'Offer Accepted'])
submis.to_csv('submission.csv', index=False)
su = pd.read_csv('submission.csv')
su.head()

,id,Offer Accepted
0,0,Yes
1,1,Yes
2,2,No
3,3,No
4,4,Yes


# TO DO
- Have a look at the test and train distribution. (Although it will not be of much help unless you fix accuracy of the training set here).

- Study what is happening to the misclassified point.

*Note: SMOTE made the accuracy worse by one percentage. However, it gave 69% on one fold of crossvalidation for RFC, implying there is a subset of training data that can be used to improve accuracy of the model*
